# CONVERT FILES WITH DUCK_DB

In [1]:
import pandas as pd
import duckdb
import os
import time

In [2]:
def time_it(func):
    """Decorator to measure the execution time of a function."""
    def wrapper(*args, **kwargs):
        start_time = time.time()  
        result = func(*args, **kwargs) 
        end_time = time.time() 
        elapsed_time = end_time - start_time  
        print(f"Execution time: {elapsed_time:.4f} seconds")  
        return result
    return wrapper

## Ownership history

In [3]:
# path = "E:\dati_moody\ownership_history\links_2022\part-00000-8f9cac6d-cf88-4461-91b4-13c784cdf6a9-c000.snappy.parquet" 

In [4]:
eu27_countries = [
    "AT", "BE", "BG", "HR", "CY", "CZ", 
    "DK", "EE", "FI", "FR", "DE", "GR", 
    "HU", "IE", "IT", "LV", "LT", "LU", 
    "MT", "NL", "PL", "PT", "RO", "SK", 
    "SI", "ES", "SE"
]

In [5]:

TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\*.parquet" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_raw\\key_financials\\key_financials_eur\\*.parquet"


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")
            
    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            {year} AS year,
            key_financials.number_of_employees AS guo_number_of_employees,
            key_financials.closing_date AS guo_closing_date,
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN (
            SELECT 
                number_of_employees, 
                closing_date,
                bvd_id_number,
                EXTRACT(YEAR FROM closing_date) AS financial_year  -- Extract year from closing_date
            FROM 
                '{TEMP_TABLE_KEY_FINANCIALS}' 
        ) AS key_financials
        ON main.subsidiary_bvd_id = key_financials.bvd_id_number
        AND EXTRACT(YEAR FROM key_financials.closing_date) = {year}  -- Directly use {year} in the join condition
        WHERE 
            main."type_of_relation" = 'GUO 25'
            AND main."guo_25" LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df


def convert_to_stata(df, output_path, country, year):
    os.makedirs(output_path, exist_ok=True)
    df.to_stata(f"{output_path}\\{country}_{year}.dta", write_index=False)


@time_it
def fetch_and_convert_to_stata(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    convert_to_stata(df, output_path, country, year)
    print(f"Data for {country} - {year} has been converted to Stata.")

### Test

In [6]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "AT"
year = "2007"
path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
output_path = r"..\data_processed\\guo_subs_europee\\geo_guos_eu"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
df

# fetch_and_convert_to_stata(country=country, year=year, path=path, output_path=output_path)

AT - 2007...


,subsidiary_bvd_id,guo_25,subsidiary_nuts2,subsidiary_nace4,guo_nuts2,guo_nace4,guo_type_of_entity,guo_status,year,guo_number_of_employees,guo_closing_date
0,DE5070188491,AT9110283772,DEA1 - Duesseldorf,6619,AT13 - Wien,None,Corporate,Active,2007,NaN,2007-12-31
1,AT9110279996,AT9110279996,None,7210,None,7210,Corporate,Dissolved,2007,NaN,2007-12-31
2,DE5070188491,AT9110283772,DEA1 - Duesseldorf,None,AT13 - Wien,None,Corporate,Active,2007,NaN,2007-12-31
3,DE5070188491,AT9110283772,DEA1 - Duesseldorf,None,AT13 - Wien,None,Corporate,Active,2007,NaN,2007-12-31
4,DE5070188491,AT9110283772,DEA1 - Duesseldorf,None,AT13 - Wien,None,Corporate,Active,2007,NaN,2007-12-31
...,...,...,...,...,...,...,...,...,...,...,...
511342,AT9130116522,AT9130116522,None,None,None,None,None,None,2007,NaN,2007-12-31
511343,GB01855172,AT*1525755171,None,None,None,None,None,None,2007,NaN,2007-12-31
511344,GB01855172,AT*1525755171,None,None,None,None,None,None,2007,NaN,2007-12-31
511345,GB02736112,AT*1525755171,None,None,None,None,None,None,2007,NaN,2007-12-31


In [7]:
df.guo_nuts2.isna().sum()

np.int64(351814)

In [ ]:
import time
import json

errors = {}
output_path = r"..\data_processed\\guo_subs_europee\\geo_guos_eu"

total_start = time.time()

for country in eu27_countries:
    for year in range(2007, 2022):
        year = str(year)
        try:
            start_time = time.time()
            
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_stata(year, country, path, output_path)
            
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue

total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

# Save errors t a JSON file
with open("errors.json", "w") as f:
    json.dump(errors, f)


AT - 2007...
Data for AT - 2007 has been converted to Stata.
Execution time: 59.7066 seconds
Data for AT - 2007 has been converted to Stata.
AT - 2008...
Data for AT - 2008 has been converted to Stata.
Execution time: 57.0270 seconds
Data for AT - 2008 has been converted to Stata.
AT - 2009...
Data for AT - 2009 has been converted to Stata.
Execution time: 58.3968 seconds
Data for AT - 2009 has been converted to Stata.
AT - 2010...
Data for AT - 2010 has been converted to Stata.
Execution time: 59.8916 seconds
Data for AT - 2010 has been converted to Stata.
AT - 2011...
Data for AT - 2011 has been converted to Stata.
Execution time: 64.0330 seconds
Data for AT - 2011 has been converted to Stata.
AT - 2012...


In [ ]:
# open errors
with open("errors.json", "r") as f:
    errors = json.load(f)

errors

## Firmographics

In [ ]:
# key = pd.read_parquet("key_financials\\key_financials_eur\part-00000-d5e63738-ec0f-45f1-9e6e-e90c27a22308-c000.snappy.parquet")
path = "..\\data_raw\\key_financials\\key_financials_eur\\part-00000-d5e63738-ec0f-45f1-9e6e-e90c27a22308-c000.snappy.parquet"

query = f"""
    SELECT 
        bvd_id_number number_of_employees, closing_date
    FROM 
        '{path}'
"""

# Execute the query
conn = duckdb.connect()
df = conn.execute(query).fetchdf()
df

,number_of_employees,closing_date
0,1.0,2021-12-31
1,2.0,2022-12-31
2,1.0,2018-12-31
3,27.0,2018-12-31
4,85.0,2018-12-31
...,...,...
5012475,2.0,2016-12-31
5012476,2.0,2015-12-31
5012477,2.0,2014-12-31
5012478,2.0,2015-12-31


In [11]:
df.columns

Index(['bvd_id_number', 'consolidation_code', 'filing_type', 'closing_date',
       'number_of_months', 'audit_status', 'accounting_practice',
       'source_for_publicly_quoted_companies_', 'original_units',
       'original_currency', 'exchange_rate_from_original_currency',
       'operating_revenue_turnover_', 'p_l_before_tax',
       'p_l_for_period_net_income_', 'cash_flow', 'total_assets',
       'shareholders_funds', 'current_ratio_x_', 'profit_margin_',
       'roe_using_p_l_before_tax_', 'roce_using_p_l_before_tax_',
       'solvency_ratio_asset_based_', 'price_earning_ratio_x_',
       'number_of_employees', 'market_capitalisation_mil_'],
      dtype='object')

In [ ]:

query = f"""
    SELECT *
    FROM 
        '{nuts_path}'
"""

# Execute the query
conn = duckdb.connect()
df = conn.execute(query).fetchdf()

## Key Financials

In [ ]:

key = pd.read_parquet("key_financials\\key_financials_eur\part-00000-d5e63738-ec0f-45f1-9e6e-e90c27a22308-c000.snappy.parquet")

: 

In [48]:
#key[['bvd_id_number', 'number_of_employees']]
key['year'] = key['bvd_id_number'] + key['consolidation_code'] + key['filing_type'] + str(key['closing_date'])

In [ ]:
# look for duplicates

key

,bvd_id_number,consolidation_code,filing_type,closing_date,number_of_months,audit_status,accounting_practice,source_for_publicly_quoted_companies_,original_units,original_currency,...,shareholders_funds,current_ratio_x_,profit_margin_,roe_using_p_l_before_tax_,roce_using_p_l_before_tax_,solvency_ratio_asset_based_,price_earning_ratio_x_,number_of_employees,market_capitalisation_mil_,year
0,CN9463468227,LF,Local registry filing,2021-12-31,12,None,None,None,units,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,CN9463468227LFLocal registry filing0 2...
1,CN9463468187,LF,Local registry filing,2022-12-31,12,None,None,None,units,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,CN9463468187LFLocal registry filing0 2...
2,CN9463468049,LF,Local registry filing,2018-12-31,12,None,None,None,units,CNY,...,-765.0,NaN,NaN,NaN,NaN,-17.65,NaN,1.0,NaN,CN9463468049LFLocal registry filing0 2...
3,CN9463467866,LF,Local registry filing,2018-12-31,12,None,None,None,units,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN,CN9463467866LFLocal registry filing0 2...
4,CN9463467865,LF,Local registry filing,2018-12-31,12,None,None,None,units,CNY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,NaN,CN9463467865LFLocal registry filing0 2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5012475,RU56826278,U1,Local registry filing,2016-12-31,12,None,Local GAAP,None,thousands,RUB,...,163564.0,NaN,NaN,-0.71,0.0,58.63,NaN,2.0,NaN,RU56826278U1Local registry filing0 202...
5012476,RU56826278,U1,Local registry filing,2015-12-31,12,None,Local GAAP,None,thousands,RUB,...,128183.0,NaN,NaN,-0.43,0.0,57.01,NaN,2.0,NaN,RU56826278U1Local registry filing0 202...
5012477,RU56826278,U1,Local registry filing,2014-12-31,12,None,Local GAAP,None,thousands,RUB,...,177693.0,NaN,NaN,-0.21,0.0,67.94,NaN,2.0,NaN,RU56826278U1Local registry filing0 202...
5012478,RU56849919N,LF,Local registry filing,2015-12-31,12,None,None,None,units,RUB,...,113.0,1.01,2.44,411.11,NaN,0.77,NaN,2.0,NaN,RU56849919NLFLocal registry filing0 20...


In [ ]:
#str(key[['year']].iloc[0])[-50:]

'gistry filing0         2...\nName: 0, dtype: object'

In [50]:
key[['closing_date']]

,closing_date
0,2021-12-31
1,2022-12-31
2,2018-12-31
3,2018-12-31
4,2018-12-31
...,...
5012475,2016-12-31
5012476,2015-12-31
5012477,2014-12-31
5012478,2015-12-31


In [ ]:
"""
e ricorda che: A financial year of accounts  is identified by the key: BVDID + Consolidation code + Filing type + Closing date. Up to 4 accounts can be delivered by company by year
Financials are available for a maximum period of 10 years

There are 3 tables (all with the same structure) in the key_financials database i.e.
1) key_financials (values are denominated in Original currency)
2) key_financials_usd (values are denominated in US dollar)
3) key_financials_eur (values are denominated in Euro)
"""